## Requisitos

Bibliotecas não usuais para se executar esse notebook

In [ ]:
!pip install rdflib

## Inputs

Precisamos que o usuário insira esses dados

In [ ]:
steps = 2 #quantidade máxima de nós que serão percorridos em profundidade
start = 'http://pt.dbpedia.org/resource/Tecido_mole' #string que indica qual será a primeira página acessada
stop = 'http://pt.dbpedia.org/resource/Laço_(teoria_dos_grafos)' #string que indica qual será a página alvo

## Extraindo os Dados

Nesse projeto trabalharemos com dados da [DBPedia em português](http://pt.dbpedia.org/), portanto o primeiro passo após o input do usuário é retirar os dados da nossa fonte, que vem em formato de RDF (NTriples)

In [ ]:
import requests as r
import rdflib
import sys
from unicodedata import category
import urllib


#Lista de pontuações existentes
punctuation =  [chr(i) for i in range(sys.maxunicode) if category(chr(i)).startswith("P") or category(chr(i)).startswith("S")] + [' '] 
numeros_por_extenso = {1:'um', 2:'dois', 3:'três', 4:'quatro', 5:'cinco', 6:'seis', 7:'sete', 8:'oito', 9:'nove', 0:'zero'}

In [ ]:
triple = rdflib.Graph()
raw_start = start.split('/')[-1]
triple.load(f'http://pt.dbpedia.org/resource/{urllib.parse.quote_plus(raw_start)}') #Suporte para caracteres brasileiros como ç

In [ ]:
#Função para deixar o link da wikipedia de um jeito que possa ser lido pelo Neo4J

def object_treatment(objeto, replace):
  #Criando um nome menor para os atributos, já que quase todos são urls fazer um split por "/" nos faz pegar o nome da página
  raw_obj = objeto.split('/')[-1]

  #Limitando o Tamanho dos Objetos para que seja possível construir o grafo
  if len(raw_obj) > 40:
    raw_obj = raw_obj[:40]
      
  if len(objeto) > 256:
    objeto = objeto[:256]

  #Se o atributo for um número
  if any(char.isnumeric() for char in raw_obj):
    for char in raw_obj:
      if char.isnumeric():
        raw_obj = raw_obj.replace(char, numeros_por_extenso[int(char)])

  #Verificando se o nome e o objeto são válidos e atribuindo seu tipo, seja ele uma página ou atributo
  if '' != objeto and '' != raw_obj:

    #Excluindo caracres especiais do nome
    if any([True if (a in raw_obj) else False for a in punctuation]):
      for p in punctuation:
        raw_obj = raw_obj.replace(p,replace)

    #Observando se o texto está em alfabeto latino <- Tive muito problema com idiomas diferentes como birmanes e chines
    try:
      raw_obj.encode('iso-8859-1')
      return raw_obj, 'Page' if ('http' in objeto or 'dbpedia' in objeto) else 'Atribute'
    except:
      return -1, -1

  return -1 , -1
  
    

In [ ]:
def busca_coneccoes(script, triple, ja_criados, coneccoes):

  for sujeito, predicado, objeto in triple:

    objeto = objeto.replace('\'', '').replace('\"', '')
    object_treated, tipo = object_treatment(objeto, '_')
    
    if object_treated == -1:
      continue
    
    #Observando se já foi criado um nó com o mesmo nome para não existir duplicidade
    if object_treated not in ja_criados.keys():    
      ja_criados[object_treated] = objeto  

      #Printando o Script de criação dos demais nós
      script.write("CREATE (")
      script.write(f"{object_treated.capitalize()}:{tipo} ")
      script.write("{")
      script.write(f"name:\'{object_treated.capitalize()}\',")
      script.write(f"url:\'{objeto}\'" if tipo == 'Page' else f"info:\'{objeto}\'")
      script.write("})\n")

      coneccoes.append([object_treatment(sujeito, '_')[0], object_treatment(predicado.split('/')[-1], '')[0], object_treated])
  
  return ja_criados, coneccoes

In [ ]:
def generate_dict(coneccoes, ja_processados, alvo):
  conect_dict = {}
  lista = []

  for s, p, o in coneccoes:
    if s == alvo:
      lista.append(o)
  
  conect_dict[alvo] = lista
  ja_processados.append(alvo)

  return conect_dict, ja_processados

In [ ]:
triple = rdflib.Graph()
triple.load(start)

ja_criados = {}
ja_processados = []
coneccoes = []
profundidade = 0

script = open('script.cypher', 'w')

#Printando o script de criação do primeiro nó
script.write("CREATE (")
script.write(f"{raw_start.capitalize()}:Page ")
script.write("{")
script.write(f"name:\'{raw_start.capitalize()}\', url:\'{start}\', start:\'True\'")
script.write("})\n")

ja_criados[raw_start] = start

ja_criados, coneccoes = busca_coneccoes(script, triple, ja_criados, coneccoes)

while profundidade < steps:
  for elem in ja_criados.copy().keys():
    if elem not in ja_processados:
      
      conect_dict, ja_processados = generate_dict(coneccoes, ja_processados, elem)
  
      for obj in conect_dict.keys():
        try:
          print(elem, end = '->')
          triple = rdflib.Graph()
          triple.load(ja_criados[obj])
          ja_criados, coneccoes = busca_coneccoes(script, triple, ja_criados, coneccoes)
          print('OK')
        except:
          print('Não foi encontrado')
          pass
        

  profundidade += 1

if coneccoes != []:
  script.write("CREATE ")
  for n in range(0,len(coneccoes)):
    
    text = f"({coneccoes[n][0]})-[:{coneccoes[n][1]}]->({coneccoes[n][2]})"
    text =  text + ',' if n != len(coneccoes) - 1 else text #Adiciona uma vírgula se não for o último elemento
    script.write(text + '\n')